Import Essential Libraries

In [1]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import numpy as np

Load the data and Check the info of data

In [2]:
data=pd.read_csv('uci-secom.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1567 entries, 0 to 1566
Columns: 592 entries, Time to Pass/Fail
dtypes: float64(590), int64(1), object(1)
memory usage: 7.1+ MB


In [3]:
data.shape

(1567, 592)

In [4]:
data.head()

,Time,0,1,2,3,4,5,6,7,8,...,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,...,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,...,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,...,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1
3,2008-07-19 14:43:00,2988.72,2479.90,2199.0333,909.7926,1.3204,100.0,104.2367,0.1217,1.4882,...,73.8432,0.4990,0.0103,0.0025,2.0544,0.0202,0.0149,0.0044,73.8432,-1
4,2008-07-19 15:22:00,3032.24,2502.87,2233.3667,1326.5200,1.5334,100.0,100.3967,0.1235,1.5031,...,NaN,0.4800,0.4766,0.1045,99.3032,0.0202,0.0149,0.0044,73.8432,-1


The data consists of  1567 entries with 592 features, which it would mean

It is mentioned that this dataset consists of null value, let's check how many columns consist of null value!

In [5]:
np.count_nonzero(data.isnull().sum())

538

There is 538 columns with consisting of null values, the quantity is quite high, let's check the highest null values contained in our columns.

In [10]:
no_na=data.isnull().sum()
pd.DataFrame(data=no_na,index=data.columns)

,0
Time,0
0,6
1,7
2,14
3,14
4,14
5,14
6,14
7,9
8,2


In [11]:
pd.set_option('display.max_rows',None)
no_na.sort_values(ascending=False)

158          1429
292          1429
293          1429
157          1429
85           1341
492          1341
220          1341
358          1341
517          1018
245          1018
246          1018
518          1018
382          1018
516          1018
109          1018
111          1018
244          1018
110          1018
384          1018
383          1018
579           949
581           949
580           949
578           949
346           794
345           794
73            794
72            794
519           715
385           715
112           715
247           715
568           273
566           273
565           273
569           273
563           273
562           273
567           273
564           273
555           260
553           260
554           260
557           260
556           260
549           260
546           260
548           260
547           260
550           260
551           260
552           260
90             51
89             51
363            51
362       

Wow!Seem like some of the columns are almost empty!We will take care of these features shortly. Let's set our evaluation model and metrics first.

Since some of the columns will be removed in every step, which leads to loss of information. so, it's important to check the perfomance of our model before removing them so that we can get a sense of whether we are going in the right direction or not.

I'm going to use a simple logistic regressor as my classifier.

Before implementing the classifier, it is important to make sure all of our features consist of only numerical data.
We could check this by referring to the datatype of each features.

In [10]:
typedata={str(k): len(list(v)) for k, v in data.groupby(data.dtypes, axis=1)}
typedata

{'int64': 1, 'float64': 590, 'object': 1}

In [8]:
bygroup=data.columns.to_series().groupby(data.dtypes).groups
bygroup

{int64: ['Pass/Fail'], float64: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', ...], object: ['Time']}

In [9]:
data.select_dtypes(include=['object']).columns

Index(['Time'], dtype='object')

In [15]:
data['Time']

0       2008-07-19 11:55:00
1       2008-07-19 12:32:00
2       2008-07-19 13:17:00
3       2008-07-19 14:43:00
4       2008-07-19 15:22:00
5       2008-07-19 17:53:00
6       2008-07-19 19:44:00
7       2008-07-19 19:45:00
8       2008-07-19 20:24:00
9       2008-07-19 21:35:00
10      2008-07-19 21:57:00
11      2008-07-19 22:52:00
12      2008-07-20 03:35:00
13      2008-07-21 08:21:00
14      2008-07-21 11:53:00
15      2008-07-22 00:03:00
16      2008-07-22 02:59:00
17      2008-07-22 08:41:00
18      2008-07-22 11:47:00
19      2008-07-22 14:00:00
20      2008-07-22 15:30:00
21      2008-07-23 05:15:00
22      2008-07-23 19:22:00
23      2008-07-25 15:23:00
24      2008-07-27 04:18:00
25      2008-07-27 09:37:00
26      2008-07-27 11:10:00
27      2008-07-27 15:46:00
28      2008-07-27 16:06:00
29      2008-07-27 16:49:00
30      2008-07-27 20:24:00
31      2008-07-27 22:28:00
32      2008-07-27 22:28:00
33      2008-07-27 23:14:00
34      2008-07-28 03:31:00
35      2008-07-28 0

The time column is referring to the specific test point, which it may not influence much on the result. Semiconductor testing is strongly depending on the testing program itself.
Unless, there is some abnormality of testing program on that specific time range. This is abnormal and unusual to occur!  And so, we should ignore the timestamp.
Let's drop the column!

In [16]:
data.drop('Time',axis=1,inplace=True)

In [17]:
def percentna(dataframe, threshold):
    columns = dataframe.columns[(dataframe.isna().sum()/dataframe.shape[1])>threshold] ## return columns which is True
    return columns.tolist()

In [27]:
na_columns = percentna(data, 0.5)
len(na_columns)

32

There is total 32 columns which consist of more than 50% null values. They are useless. Let.s get rid of the columns!

0            False
1            False
2            False
3            False
4            False
5            False
6            False
7            False
8            False
9            False
10           False
11           False
12           False
13           False
14           False
15           False
16           False
17           False
18           False
19           False
20           False
21           False
22           False
23           False
24           False
25           False
26           False
27           False
28           False
29           False
30           False
31           False
32           False
33           False
34           False
35           False
36           False
37           False
38           False
39           False
40           False
41           False
42           False
43           False
44           False
45           False
46           False
47           False
48           False
49           False
50           False
51           False
52          